# Robust linear regression: An exploration

In the previous lesson, we saw that an outlier can lead a model with a Normal likelihood function in the wrong direction. Normal distributions have very light tails and any model that uses them needs to compensate for outliers by disproportionately adjusting the values of the Normal mean and variance in the posterior distribution.

The goal of robust regression is to either reduce the impact of outliers on the posterior mean and variance or to explicitly detect and identify outliers. Another common practice is to remove outliers manually but _that_ road is fraught with bias. We should always encode our assumptions _in the model_ rather than modifying the data on an ad hoc basis.

## Prep plan

* There is no assigned reading from the textbook today.
* You will spend all your prep time on this workbook.


## Workbook outline

1. We start by reproducing the results from the last class where we did linear regression with a Normal likelihood function.

2. We then explore how a heavy-tailed distribution in the likelihood function changes the inference results.

3. Finally, we look at a more complicated model that automatically detects which data are outliers and which are not.

**Click the _Run All_ button now** so the workbook produces output while you are reading the information below.

## Dataset

As before, this data set shows relates the percentage of adults who do not engage in any physical exercise/activity to the percentage of adults who are obese in different states in the USA. Source: [Huston, J. (2022, January 23). Lab 13 Obesity. Kaggle code.](https://www.kaggle.com/code/joshuahuston/lab-13-obesity/notebook)

In order to test how well our models work, we create two copies of the data set — one with and one without the outlier.

To be clear, we want to fit our models to the complete data set but we also want that the outlier in the complete data set doesn’t affect our results much. In order to check whether or not the outlier affects our results, we check what happens if we remove the outlier manually.

**Run this code cell** and look at the plots of the two copies of the data.

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import scipy.stats as sts


df = pd.read_csv(
    'https://course-resources.minerva.edu/uploaded_files/mu/00265107-3137/obesity.csv')

# The complete data set. We sort the data by x-coordinate to make the plots look
# better so we know datum number 3, for example, is the 3rd point from the left.
# This has no impact on the inference results.
data_x = np.array(df['no_activity_perc'])
order = np.argsort(data_x)
data_x = data_x[order]
data_y = np.array(df['obesity_perc'])[order]

# Remove the outlier and create a second data set
outlier = np.where(data_x > 40)[0][0]
data_removed_x = np.concatenate((data_x[:outlier], data_x[outlier+1:]))
data_removed_y = np.concatenate((data_y[:outlier], data_y[outlier+1:]))

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.title('Complete data set')
plt.xlabel('% adults reporting no leisure physical activity')
plt.ylabel('% adults with obesity')
plt.plot(data_x, data_y, 'k.', alpha=0.5)
plt.axis('equal')

plt.subplot(1, 2, 2)
plt.title('Data set with outlier removed')
plt.xlabel('% adults reporting no leisure physical activity')
plt.ylabel('% adults with obesity')
plt.plot(data_removed_x, data_removed_y, 'k.', alpha=0.5)
plt.axis('equal')

plt.show()

Explain in your own words _how much_ of a difference that one outlier could make to the results. You should discuss why it is an outlier (how do you see that on the plot) and how far away it is from what you would expect of a non-outlier.

In [ ]:
# ENTER YOUR ANSWER IN YOUR FORUM PRE-CLASS WORKBOOK

## The Normal model (same as last time)

We fit the model from the previous lesson with and without the outlier data point.

Likelihood:

$$y_i \sim \text{Normal}(\mu_i, \sigma^2)$$

$$\mu_i = c_0 + c_1 x_i$$

Prior:

$$c_0 \sim \text{Uniform}(0, 100)$$

$$c_1 \sim \text{Normal}(0, 5^2)$$

$$\sigma \sim \text{Uniform}(0, 100)$$

**Run this code cell** to create the model in PyMC.

In [ ]:
with pm.Model() as normal_model:
    # Prior
    c0 = pm.Uniform('c0', lower=0, upper=100)
    c1 = pm.Normal('c1', mu=0, sigma=5)
    sigma = pm.Uniform('sigma', lower=0, upper=100)
    # Data
    x = pm.Data('x', data_x)
    y = pm.Data('y', data_y)
    # Regression mean
    mu = pm.Deterministic('mu', c0 + c1 * x)
    # Likelihood
    pm.Normal('likelihood', mu=mu, sigma=sigma, observed=y)

from IPython.display import Image
Image(pm.model_to_graphviz(normal_model).render(format='png'))

This model has 3 free variables — $c_0$, $c_1$, and $\sigma$. Explain why we do not count the $x_i$, $y_i$ and $\mu_i$ variables as _free_ variables.

In [ ]:
# ENTER YOUR ANSWER IN YOUR FORUM PRE-CLASS WORKBOOK

### Normal model: Posteriors with all data

We run the sampler on the full data set, contained in the variables `data_x` and `data_y`.

**Run this code cell** and confirm that the diagnostic output from the sampler looks fine.

* The R-hat values should be close to 1.
* The effective sample size should be at least a few hundred.
* The rank-plot histograms should be approximately uniform.

**Note:** We don't display _all_ the posterior variables in the diagnostic output since there are too many of them — there is a $\mu_i$ parameter for each data point. We just display the linear regression coefficients ($c_0$ and $c_1$) and the noise scale ($\sigma$). The `var_names` argument is used to specify which variables are shown.

In [ ]:
with normal_model:
    normal_inference_all = pm.sample()

az.plot_rank(normal_inference_all, var_names=['c0', 'c1', 'sigma'])
az.summary(normal_inference_all, var_names=['c0', 'c1', 'sigma'])

### Normal model: Posteriors without the outlier

We run the sampler without the outlier datum. Note how the code below uses the `set_data` function to update the values of the PyMC variables `x` and `y`.

**Run this code cell** and confirm that the diagnostic output from the sampler looks fine.

In [ ]:
with normal_model:
    pm.set_data({
        'x': data_removed_x,
        'y': data_removed_y})
    normal_inference_without = pm.sample()

az.plot_rank(normal_inference_without, var_names=['c0', 'c1', 'sigma'])
az.summary(normal_inference_without, var_names=['c0', 'c1', 'sigma'])

### The difference

To see that the posteriors changed quite a lot, we can look at the `az.summary()` output for `c0`, `c1`, and `sigma`. However, it is easier to see on a plot.

**Run the code cell below** and interpret the results on the plots.

In [ ]:
plt.figure(figsize=(12, 4))
plt.suptitle('Posterior distributions for the two data sets')
for i, var in enumerate(['c0', 'c1', 'sigma']):
    plt.subplot(1, 3, i+1)
    plt.xlabel(var)
    if i == 0:
        plt.ylabel('probability density')
    plt.hist(getattr(normal_inference_all.posterior, var).values.flatten(), density=True, bins=50, edgecolor='white', alpha=0.5, label='all data')
    plt.hist(getattr(normal_inference_without.posterior, var).values.flatten(), density=True, bins=50, edgecolor='white', alpha=0.5, label='no outlier')
plt.legend()
plt.show()

Interpret the results. How much of a difference does the outlier data point make in the posteriors over $c_0$, $c_1$, and $\sigma$? Is this a big difference or a small difference? Explain.

In [ ]:
# ENTER YOUR ANSWER IN YOUR FORUM PRE-CLASS WORKBOOK

## Student T linear regression

A problem with the Normal distribution in the likelihood function is that its tails are too light to deal with outliers. That means the probability of observing what we usually call an outlier is really, really low according to the Normal likelihood. As a result, the posterior distribution needs to make $\sigma$ larger to accommodate the outlier. Usually the posterior also needs to adjust $c_0$ and $c_1$ to make the straight line fit tend towards the outlier.

Instead of the light-tailed Normal distribution, we can attempt to use a distribution with heavier tails. This is a common approach to what is called _robust_ linear regression — robust here means that our inference about the regression line is not overly influenced by a small number of outliers. Note that this changes the likelihood function for all the data and not just the outlier. A model with a heavy-tailed likelihood basically assumes that the data distribution can produce samples far away from the mean.

The T distribution, also known as the Student T distribution, has a parameter called $\nu$ (pronounced "new") that adjusts the lightness of the tails. The larger $\nu$ is, the lighter the tails are.
* When $\nu=1$, the tails are very heavy and samples that look like outliers are common.
* When $\nu\approx30$, the T distribution already looks a lot like the Normal distribution and you can prove that for $\nu\rightarrow\infty$, the T distribution converges to the Normal distribution.

That's really nice. If there are no outliers in the data, a linear regression model with a T likelihood should give you virtually the same results as a model with a Normal likelihood since $\nu$ should be large in the posterior. However, if there are outliers, the model should automatically make $\nu$ small, resulting in heavier tails.

### Example: How far away are typical samples from the mean?

Here is a quick demonstration of how a heavy-tailed T distribution really is very different from a light-tailed Normal distribution. If we generate 100 samples from a standard Normal distribution and find the maximum, it's rarely more than 3 sigma units away from the mean. For the standard normal, the mean is 0 and $\sigma=1$.

If you run the code cell below, which generates a hundred $\text{Normal}(0, 1)$ samples, a few times, you should see that the maximum value is usually between 2 and 3 and only rarely more than 3.

In [ ]:
sts.norm(loc=0, scale=1).rvs(100).max()

Let's try that with a T distribution with $\nu=1$, which is heavy-tailed. It is not uncommon to get samples that are _hundreds_ of sigmas away from the middle. (The probability of seeing a maximum greater than 100 is about 0.28.) If you run the cell lots of times, you might even get a result that is _thousands_ of sigmas from the middle. (The probability of seeing a maximum greater than 1000 is about 0.03).

We would never observe such an outcome with the Normal distribution, even if we waited until the heat death of the universe. The probability of seeing a maximum greater than 100 with Normal samples is approximately $10^{-120}$.

You should run the code cell below for different values of $\nu$ (the `df` parameter in the code since $\nu$ is known as the "degrees of freedom" parameter) and confirm that the samples behave much more like Normal samples when $\nu\ge30$.

In [ ]:
sts.t.rvs(df=1, loc=0, scale=1, size=100).max()

You can also experiment further with the T distribution using [this Geogebra plot](https://www.geogebra.org/m/za632sqy) or [this interactive plot](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html). Note how wide the 95% interval of the distribution is when $\nu=1$.

You can, of course, also plot the T distribution in Python. The T distribution has a lot more probability mass away from the middle (in the tails).

In [ ]:
plt.figure(figsize=(8, 4))
plt.title(r'PDF of the T distribution with $\nu=1$')
plt.ylabel('probability density')
plot_x = np.linspace(-5, 5, 500)
plt.plot(plot_x, sts.t.pdf(plot_x, df=1, loc=0, scale=1), label='T')
plt.plot(plot_x, sts.norm.pdf(plot_x, loc=0, scale=1), label='Normal(0,1)')
plt.legend()
plt.xlim(-5, 5)
plt.ylim(0, plt.ylim()[1])
plt.show()

## The Student T model

Modified likelihood:

$$\color{blue}{y_i \sim \text{T}(\nu, \mu_i, \sigma)}$$

Everything else stays the same, except that we require a prior for $\nu$.

$$\mu_i = c_0 + c_1 x_i$$

Prior:

$$c_0 \sim \text{Uniform}(0, 100)$$

$$c_1 \sim \text{Normal}(0, 5^2)$$

$$\sigma \sim \text{Uniform}(0, 100)$$

$$\color{blue}{\nu \sim \text{Half-Normal}(\sigma=30)}$$

The prior over $\nu$ is somewhat arbitrary. It is the positive half of a $\text{Normal}(0, 30^2)$ distribution (since $\nu>0$) with the scale parameter set to 30 so the prior allows for values of $\nu$ that make the T distribution look like the Normal distribution. In this way, the model can adapt to whether or not there are outliers.

In [ ]:
with pm.Model() as t_model:
    # Prior
    c0 = pm.Uniform('c0', lower=0, upper=100)
    c1 = pm.Normal('c1', mu=0, sigma=5)
    sigma = pm.Uniform('sigma', lower=0, upper=100)
    nu = pm.HalfNormal('nu', sigma=30)  # <== THIS LINE IS NEW
    # Data
    x = pm.Data('x', data_x)
    y = pm.Data('y', data_y)
    # Regression mean
    mu = pm.Deterministic('mu', c0 + c1 * x)
    # Likelihood
    pm.StudentT('likelihood', nu=nu,  # <== THIS LINE IS DIFFERENT
                mu=mu, sigma=sigma, observed=y)

from IPython.display import Image
Image(pm.model_to_graphviz(t_model).render(format='png'))

If there are no outliers in the data, what do you expect the posterior distribution over $\nu$ to look like?

In [ ]:
# ENTER YOUR ANSWER IN YOUR FORUM PRE-CLASS WORKBOOK

### Student T model: Posteriors with all data

We compute posteriors for the full data set and then for the data set excluding the outlier, to see how much the outlier affects the results.

**Run this code cell** and check the diagnostic output.

In [ ]:
with t_model:
    t_inference_all = pm.sample()

az.plot_rank(t_inference_all, var_names=['c0', 'c1', 'sigma', 'nu'])
az.summary(t_inference_all, var_names=['c0', 'c1', 'sigma', 'nu'])

**Run this code cell** to make a pair plot of the posterior distribution over $(c_0, c_1, \sigma, \nu)$.

In [ ]:
ax = az.plot_pair(
    t_inference_all,
    var_names=['c0', 'c1', 'sigma', 'nu'],
    marginals=True,
    kind=["scatter", "kde"],
    scatter_kwargs={"color": "C0", "alpha": 0.05},
    marginal_kwargs={"kind": "kde", "color": "C0"},
    kde_kwargs={"contour_kwargs": {"colors": "k", 'alpha': 1}});

This is a somewhat complicated 4-dimensional posterior distribution. You should spend some time looking at each pair plot to work out how the variables interact in the posterior.

As with the Normal likelihood model, we see a strong negative correlation between $c_0$ and $c_1$.

Briefly explain why this negative correlation exists.

In [ ]:
# ENTER YOUR ANSWER IN YOUR FORUM PRE-CLASS WORKBOOK

There don't seem to be strong interactions (correlations) between $\sigma$ and either $c_0$ or $c_1$.

An interesting (and new) feature of the posterior distribution is the interaction between $\nu$ and the other variables. Look at the $\sigma$-$\nu$ plot (last row, third column), for example.
* If $\nu$ is small (close to 0), the most likely values for $\sigma$ are around $2$.
* If $\nu$ is large (greater than 20 or so), the possible $\sigma$ values are quite spread out but somewhere in the range $[2.3, 3.1]$.

Why, according to the posterior, does $\sigma$ need to be larger when $\nu$ is larger? Consider how the T distribution in the likelihood behaves for different values of $\nu$.

In [ ]:
# ENTER YOUR ANSWER IN YOUR FORUM PRE-CLASS WORKBOOK

### Student T model: Posteriors without the outlier

We recompute the posterior distribution with the same model and the data set that excludes the outlier.

In [ ]:
with t_model:
    pm.set_data({
        'x': data_removed_x,
        'y': data_removed_y})
    t_inference_without = pm.sample()

az.plot_rank(t_inference_without, var_names=['c0', 'c1', 'sigma', 'nu'])
az.summary(t_inference_without, var_names=['c0', 'c1', 'sigma', 'nu'])

In [ ]:
ax = az.plot_pair(
    t_inference_without,
    var_names=['c0', 'c1', 'sigma', 'nu'],
    marginals=True,
    kind=["scatter", "kde"],
    scatter_kwargs={"color": "C0", "alpha": 0.05},
    marginal_kwargs={"kind": "kde", "color": "C0"},
    kde_kwargs={"contour_kwargs": {"colors": "k", 'alpha': 1}});

### The difference

**Run the code cell below** and interpret the results on the plots to see how the posterior over `c0`, `c1`, `sigma`, and `nu` changes with/without the outlier.

In [ ]:
plt.figure(figsize=(16, 4))
plt.suptitle('Posterior distributions for the two data sets')
for i, var in enumerate(['c0', 'c1', 'sigma', 'nu']):
    plt.subplot(1, 4, i+1)
    plt.xlabel(var)
    if i == 0:
        plt.ylabel('probability density')
    plt.hist(getattr(t_inference_all.posterior, var).values.flatten(), density=True, bins=50, edgecolor='white', alpha=0.5, label='all data')
    plt.hist(getattr(t_inference_without.posterior, var).values.flatten(), density=True, bins=50, edgecolor='white', alpha=0.5, label='no outlier')
plt.legend()
plt.show()

You should see that the posteriors over $c_0$, $c_1$, and $\sigma$ for the two data sets are closer together than before. (Compare this plot to the matching plot for the Normal likelihood function.) However, there are obviously still some differences.

So, the results are _more_ robust to outliers even though they are not perfectly robust. The outlier still makes a difference.

Interpret the plot for $\nu$ (on the far right). Explain how and why the posterior histograms differ for the two data sets.

In [ ]:
# ENTER YOUR ANSWER IN YOUR FORUM PRE-CLASS WORKBOOK

## Model 3: Outlier classifier

This model is largely based on Hogg, D.W., Bovy, J., Lang, D. (2010). [Data analysis recipes: Fitting a model to data](https://arxiv.org/abs/1008.4686). arXiv:1008.4686.

In the previous two models, we used either a light-tailed or a heavy-tailed model for _all_ the values in the data set. We assumed that either all values were generated from a light-tailed Normal distribution or all values were generated from a heavy-tailed Student T distribution.

This assumption often doesn't align with what we really believe, namely that _some_ of the values come from a particular data distribution while _a few_ values are outliers that were generated in some other way — as a result of faulty measurement, for example.

The next model encodes this assumption by assigning a binary variable to each data point with the value 1 if the data point is an outlier and 0 if it is not. This 0/1 value is generated from a Bernoulli distribution with probability $p$. We do not assume a particular value for $p$ here and infer it from the data.

The likelihood is the same as the Normal model except that data points can have different variances (note the $i$ subscript of the $\sigma$ variable).

$$y_i \sim \text{Normal}(\mu_i, \sigma_i)$$

$$\mu_i = c_0 + c_1 x_i$$

$$\sigma_i = \left\{\begin{array}{ll}\sigma_{\text{in}} & \text{if } q_i = 0\\\sigma_{\text{in}}+\sigma_{\text{out}} & \text{if } q_i = 1\end{array}\right.$$

$$q_i \sim \text{Bernoulli}(p)$$

Prior:

$$c_0 \sim \text{Uniform}(0, 100)$$

$$c_1 \sim \text{Normal}(0, 5^2)$$

$$\sigma_{\text{in}} \sim \text{Uniform}(0, 100)$$

$$\sigma_{\text{out}} \sim \text{Half-Normal}(0, 30^2)$$

$$p \sim \text{Uniform}(0, ½)$$

**Run this code cell** to create the model in PyMC.

In [ ]:
import pytensor.tensor as pt

with pm.Model() as outlier_model:

    # Observed variables
    x = pm.Data('x', data_x)
    y = pm.Data('y', data_y)

    # Linear regression
    c0 = pm.Uniform('c0', lower=0, upper=100)
    c1 = pm.Normal('c1', mu=0, sigma=5)
    mu = pm.Deterministic('mu', c0 + c1 * x)

    # Noise parameters for inliers and outliers
    sigma = pm.Uniform('sigma', lower=0, upper=100)
    sigma_out = pm.HalfNormal('sigma_out', sigma=30)
    sigmas = pt.as_tensor_variable([sigma, sigma + sigma_out])

    # In/out class assignment probability and indicators
    p = pm.Uniform('p', lower=0, upper=0.5)
    is_outlier = pm.Bernoulli('is_outlier', p=p, size=x.shape[0])

    pm.Normal('likelihood', mu=mu, sigma=sigmas[is_outlier], observed=y)

from IPython.display import Image
Image(pm.model_to_graphviz(outlier_model).render(format='png'))

In [ ]:
with outlier_model:
    # This runs the sampler to draw from the posterior
    trace = pm.sample(2000, tune=1000, cores=1)

# This creates a summary table of the posterior distribution
summary = az.summary(trace, var_names=["c0", "c1", "sigma", "sigma_out", "p"])

# Print the summary table
print(summary)

That's quite a complicated-looking model! Here are some important features to note.
* The $p$, $c_0$, $c_1$, $\sigma_{\text{in}}$, and $\sigma_{\text{out}}$ variables are _outside_ the plate (the rectangular box). This means there is only one copy of each of those variables. In the mathematical description of the model, these variables do not have an $i$ subscript.
* The $\mu_i$, $q_i$ (`is_outlier`), $x_i$, and $y_i$ variables are inside the plate and there are 54 (the number of data) copies of them.
* We have more unknown parameters than we have data! There are only 54 data but we have 54 (the $q_i$) + 5 ($p$, $c_0$, $c_1$, $\sigma_{\text{in}}$, and $\sigma_{\text{out}}$) = 59 free variables. Note that the $\mu_i$ are not free variables since they depend deterministically on their inputs. It might seem that a model with more parameters than data is certain to overfit but this is not the case.

**Final task:** review the outlier classifier model above and make sure you understand every part of it. If something doesn't make sense, discuss it with your classmates and bring record any unresolved questions at the start of the notebook so we can discuss them in class.

We compute and discuss the posteriors for this model in class. (Feel free to compute them yourself before class if you feel like it!)